In [4]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
from src.tradecli import TradeJournal
from src.journal import journal

frozenset({'type_2_m5', 'type_1_m15', 'type_1_h4', 'type_2_h1', 'type_1_m30', 'type_1_m1', 'type_1_w', 'M', 'h1', 'w', 'h4', 'type_1_h1', 'm30', 'type_2_m15', 'm5', 'type_1_m5', 'type_1_d', 'd', 'type_2_w', 'type_1_M', 'type_2_h4', 'type_2_M', 'type_2_m30', 'type_2_d', 'm1', 'type_2_m1', 'm15'})
[Trade(uid='1', tags=[Tag(timestamp=datetime.datetime(2024, 12, 28, 16, 6, 41, 876108), key='type_1_m1', value=True), Tag(timestamp=datetime.datetime(2024, 12, 28, 16, 6, 41, 876118), key='type_3_m15', value=True), Tag(timestamp=datetime.datetime(2024, 12, 28, 16, 6, 41, 876121), key='entry_price', value=1.1), Tag(timestamp=datetime.datetime(2024, 12, 28, 16, 6, 41, 876122), key='sl_price', value=1.095), Tag(timestamp=datetime.datetime(2024, 12, 28, 16, 6, 41, 876123), key='tp_price', value=1.11)]), Trade(uid='2', tags=[Tag(timestamp=datetime.datetime(2024, 12, 28, 16, 6, 41, 876125), key='type_2_h1', value=True), Tag(timestamp=datetime.datetime(2024, 12, 28, 16, 6, 41, 876127), key='entry_pric

In [6]:
# Generate the feature dataframe
df = journal.to_dataframe()
print(df)

  trade_uid  SL_distance management_strategy type_3_m15 type_2_h1 type_1_m1  \
0         1          NaN                None       True      None      True   
1         2          NaN                None       None      True      None   
2         3          0.5                None       None      None      None   
3         4          0.5          strategy_2       None      None      None   

   close_price  entry_price  tp_price  sl_price  
0          NaN          1.1      1.11     1.095  
1          NaN          1.2      1.21     1.195  
2          NaN          1.3      1.31     1.295  
3        1.305          1.3      1.31     1.295  


In [7]:
# Example analysis: Compare trades with different SL distances
sl_trades = df[df['SL_distance'] > 0]
print(sl_trades)

  trade_uid  SL_distance management_strategy type_3_m15 type_2_h1 type_1_m1  \
2         3          0.5                None       None      None      None   
3         4          0.5          strategy_2       None      None      None   

   close_price  entry_price  tp_price  sl_price  
2          NaN          1.3      1.31     1.295  
3        1.305          1.3      1.31     1.295  
